<a href="https://colab.research.google.com/github/lgregs/TCC-CD-USP/blob/main/TCC_USP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TCC

In [1]:
!pip install kagglehub --quiet

In [2]:
# 2. Upload do kaggle.json
from google.colab import files
files.upload()  # Faça upload do arquivo kaggle.json aqui

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lukinhabalatensa","key":"030cf5f320ba4c390a5bb6b2557edec8"}'}

In [3]:
# 3. Configurar credenciais do Kaggle
import os
import shutil

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
shutil.move("kaggle.json", os.path.expanduser("~/.kaggle/kaggle.json"))
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 600)

In [4]:
# 4. Baixar dataset com kagglehub
import kagglehub

path = kagglehub.dataset_download("agrafintech/turkish-public-companies-balance-sheets-from-kap")
print("Arquivos baixados em:", path)

100%|██████████| 63.9M/63.9M [00:01<00:00, 60.6MB/s]

Extracting files...


Arquivos baixados em: /root/.cache/kagglehub/datasets/agrafintech/turkish-public-companies-balance-sheets-from-kap/versions/1


In [5]:
# 5. Listar arquivos baixados
import os
print("Arquivos disponíveis:")
print(os.listdir(path))

Arquivos disponíveis:
['Turkish Public Companies Balance Sheets from KAP']


In [6]:
# 7. Explorar conteúdo da pasta principal
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')
print("Conteúdo da pasta principal:")
print(os.listdir(main_folder))

Conteúdo da pasta principal:
['TATEN_2021.xlsx', 'ORMA_2010.xlsx', 'ERCB_2023.xlsx', 'ISYAT_2019.xlsx', 'DGATE_2023.xlsx', 'ISGYO_2024.xlsx', 'SELEC_2010.xlsx', 'KRGYO_2018.xlsx', 'GRNYO_2022.xlsx', 'BRSAN_2012.xlsx', 'BTCIM_2021.xlsx', 'KOZAA_2009.xlsx', 'CCOLA_2012.xlsx', 'VSNMD_2022.xlsx', 'BFREN_2017.xlsx', 'LYDHO_2009.xlsx', 'KUTPO_2012.xlsx', 'EPLAS_2021.xlsx', 'IHGZT_2014.xlsx', 'IHYAY_2024.xlsx', 'NUGYO_2023.xlsx', 'KIMMR_2019.xlsx', 'TCELL_2021.xlsx', 'DAGI_2012.xlsx', 'HTTBT_2022.xlsx', 'EUREN_2022.xlsx', 'TKFEN_2014.xlsx', 'KOZAA_2024.xlsx', 'GSRAY_2008.xlsx', 'TERA_2020.xlsx', 'VERUS_2019.xlsx', 'DEVA_2008.xlsx', 'SONME_2023.xlsx', 'GOODY_2008.xlsx', 'BURVA_2021.xlsx', 'TRCAS_2013.xlsx', 'LINK_2014.xlsx', 'IPEKE_2021.xlsx', 'MAALT_2019.xlsx', 'BNTAS_2022.xlsx', 'FENER_2012.xlsx', 'MARBL_2020.xlsx', 'CASA_2019.xlsx', 'SELGD_2015.xlsx', 'GRNYO_2024.xlsx', 'CCOLA_2013.xlsx', 'DESPC_2022.xlsx', 'YKSLN_2017.xlsx', 'SASA_2012.xlsx', 'GRNYO_2010.xlsx', 'SASA_2015.xlsx', 'ALKIM_201

In [7]:
import shutil

# Caminho da pasta com os arquivos Excel
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')

# Nome do arquivo zipado
shutil.make_archive('balance_sheets_dataset', 'zip', main_folder)


'/content/balance_sheets_dataset.zip'

In [8]:
from google.colab import files

# Baixar o arquivo compactado
files.download('balance_sheets_dataset.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## SQLITE

In [9]:
import sqlite3
import pandas as pd
import os

# Cria o banco SQLite
conn = sqlite3.connect('balancos.sqlite')

# Caminho da pasta com os arquivos Excel
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')
arquivos_excel = os.listdir(main_folder)

# Carrega e salva no banco
for arquivo in arquivos_excel:
    try:
        df = pd.read_excel(os.path.join(main_folder, arquivo))
        nome_tabela = os.path.splitext(arquivo)[0].lower()
        df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    except Exception as e:
        print(f"Erro ao salvar {arquivo}: {e}")

conn.close()

##

In [10]:
import os
import pandas as pd
import sqlite3
import pickle

# --- CONFIG ---
db_path = 'balancos.sqlite'
pickle_path = 'processed_files.pkl'
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')

# --- STEP 1: Load or create pickle ---
if os.path.exists(pickle_path):
    with open(pickle_path, 'rb') as f:
        processed = pickle.load(f)
    print(f"Progresso carregado com {len(processed)} arquivos já processados.")
else:
    processed = set()
    with open(pickle_path, 'wb') as f:
        pickle.dump(processed, f)
    print("Novo progresso iniciado.")

# --- STEP 2: Connect to SQLite ---
conn = sqlite3.connect(db_path)

# --- STEP 3: Process Excel files ---
arquivos_excel = os.listdir(main_folder)
print(f"Total arquivos encontrados: {len(arquivos_excel)}")

for arquivo in arquivos_excel:
    if arquivo in processed or not arquivo.endswith(('.xls', '.xlsx')):
        print(f"PULANDO: {arquivo}")
        continue
    try:
        df = pd.read_excel(os.path.join(main_folder, arquivo))
        nome_tabela = os.path.splitext(arquivo)[0].lower().replace(" ", "_").replace("-", "_")
        df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
        processed.add(arquivo)
        print(f"✅ Sucesso: {arquivo} -> tabela: {nome_tabela}")
    except Exception as e:
        print(f"❌ ERRO ao processar {arquivo}: {e}")
    finally:
        # Atualiza progresso a cada passo
        with open(pickle_path, 'wb') as f:
            pickle.dump(processed, f)

conn.close()
print("🏁 FIM: Todos arquivos processados.")


A saída de streaming foi truncada nas últimas 5000 linhas.
✅ Sucesso: KSTUR_2019.xlsx -> tabela: kstur_2019
✅ Sucesso: INTEM_2020.xlsx -> tabela: intem_2020
✅ Sucesso: NTGAZ_2022.xlsx -> tabela: ntgaz_2022
✅ Sucesso: KGYO_2013.xlsx -> tabela: kgyo_2013
✅ Sucesso: GLYHO_2016.xlsx -> tabela: glyho_2016
✅ Sucesso: BIZIM_2016.xlsx -> tabela: bizim_2016
✅ Sucesso: DOHOL_2017.xlsx -> tabela: dohol_2017
✅ Sucesso: MHRGY_2022.xlsx -> tabela: mhrgy_2022
✅ Sucesso: DAPGM_2022.xlsx -> tabela: dapgm_2022
✅ Sucesso: ATEKS_2008.xlsx -> tabela: ateks_2008
✅ Sucesso: OYAKC_2015.xlsx -> tabela: oyakc_2015
✅ Sucesso: CMBTN_2020.xlsx -> tabela: cmbtn_2020
✅ Sucesso: SMRTG_2019.xlsx -> tabela: smrtg_2019
✅ Sucesso: BSOKE_2019.xlsx -> tabela: bsoke_2019
✅ Sucesso: BIZIM_2024.xlsx -> tabela: bizim_2024
✅ Sucesso: FROTO_2021.xlsx -> tabela: froto_2021
✅ Sucesso: LUKSK_2019.xlsx -> tabela: luksk_2019
✅ Sucesso: RTALB_2023.xlsx -> tabela: rtalb_2023
✅ Sucesso: SRVGY_2019.xlsx -> tabela: srvgy_2019
✅ Sucesso: P

In [11]:
# Caminho da pasta onde estão os arquivos
main_folder = os.path.join(path, 'Turkish Public Companies Balance Sheets from KAP')

# Listar todos os arquivos Excel
arquivos_excel = os.listdir(main_folder)

# Extrair o nome da empresa de cada arquivo
empresas = [arquivo.split("_")[0] for arquivo in arquivos_excel]

# Pegar apenas empresas únicas (sem repetir)
empresas_unicas = sorted(set(empresas))

# Mostrar
print(f"Número de empresas únicas: {len(empresas_unicas)}")
print(empresas_unicas)


Número de empresas únicas: 555
['A1CAP', 'ACSEL', 'ADEL', 'ADESE', 'ADGYO', 'AEFES', 'AFYON', 'AGHOL', 'AGROT', 'AGYO', 'AHGAZ', 'AHSGY', 'AKCNS', 'AKENR', 'AKFGY', 'AKFIS', 'AKFYE', 'AKMGY', 'AKSA', 'AKSEN', 'AKSGY', 'AKSUE', 'AKYHO', 'ALARK', 'ALCAR', 'ALCTL', 'ALFAS', 'ALGYO', 'ALKA', 'ALKIM', 'ALKLC', 'ALTNY', 'ALVES', 'ANELE', 'ANGEN', 'ARASE', 'ARCLK', 'ARDYZ', 'ARENA', 'ARMGD', 'ARSAN', 'ARTMS', 'ARZUM', 'ASELS', 'ASGYO', 'ASTOR', 'ASUZU', 'ATAGY', 'ATAKP', 'ATATP', 'ATEKS', 'ATLAS', 'ATSYH', 'AVGYO', 'AVHOL', 'AVOD', 'AVPGY', 'AVTUR', 'AYCES', 'AYDEM', 'AYEN', 'AYES', 'AYGAZ', 'AZTEK', 'BAGFS', 'BAHKM', 'BAKAB', 'BALAT', 'BALSU', 'BANVT', 'BARMA', 'BASCM', 'BASGZ', 'BAYRK', 'BEGYO', 'BERA', 'BEYAZ', 'BFREN', 'BIENY', 'BIGCH', 'BIGEN', 'BIMAS', 'BINBN', 'BINHO', 'BIOEN', 'BIZIM', 'BJKAS', 'BLCYT', 'BMSCH', 'BMSTL', 'BNTAS', 'BOBET', 'BORLS', 'BORSK', 'BOSSA', 'BRISA', 'BRKO', 'BRKSN', 'BRLSM', 'BRMEN', 'BRSAN', 'BRYAT', 'BSOKE', 'BTCIM', 'BUCIM', 'BULGS', 'BURCE', 'BURVA', 'BVSA